# tons of imports most unecessary skip lol

In [27]:
# %%
import plotly
import plotly.express as px
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
import pickle

import numpy as np
import pandas as pd
import einops
from fancy_einsum import einsum
from tqdm import tqdm
import random
from pathlib import Path
# import plotly.express as px
from torch.utils.data import DataLoader
from typing import Union, List, Optional, Callable, Tuple, Dict, Literal, Set
from jaxtyping import Float, Int
from jaxtyping import Float, Int, Bool, Shaped, jaxtyped
import typeguard
from functools import partial, wraps
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.utils import to_numpy
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache, patching

import circuitsvis as cv
import os, sys

import urllib.request
import zipfile

if not os.path.exists("path_patching.py"):
    urllib.request.urlretrieve("https://github.com/callummcdougall/path_patching/archive/refs/heads/main.zip", "main.zip")
    
    with zipfile.ZipFile("main.zip", "r") as zip_ref:
        zip_ref.extract("path_patching-main/ioi_dataset.py")
        zip_ref.extract("path_patching-main/path_patching.py")
    
    sys.path.append("path_patching-main")
    
    os.remove("main.zip")
    os.rename("path_patching-main/ioi_dataset.py", "ioi_dataset.py")
    os.rename("path_patching-main/path_patching.py", "path_patching.py")
    os.rmdir("path_patching-main")


# if not os.path.exists("path_patching.py"):
#         !wget https://github.com/callummcdougall/path_patching/archive/refs/heads/main.zip
#         !unzip main.zip 'path_patching-main/ioi_dataset.py'
#         !unzip main.zip 'path_patching-main/path_patching.py'
#         sys.path.append("path_patching-main")
#         os.remove("main.zip")
#         os.rename("path_patching-main/ioi_dataset.py", "ioi_dataset.py")
#         os.rename("path_patching-main/path_patching.py", "path_patching.py")
#         os.rmdir("path_patching-main")

from path_patching import Node, IterNode, path_patch, act_patch


from neel_plotly import imshow, line, scatter, histogram
torch.set_grad_enabled(False)
device = "cuda" if torch.cuda.is_available() else "cpu"

# %%
device

# %%
update_layout_set = {
    "xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat",
    "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth", "yaxis_gridcolor",
    "showlegend", "xaxis_tickmode", "yaxis_tickmode", "xaxis_tickangle", "yaxis_tickangle", "margin", "xaxis_visible", "yaxis_visible", "bargap", "bargroupgap"
}

def imshow(tensor, return_fig = False, renderer=None, **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    facet_labels = kwargs_pre.pop("facet_labels", None)
    border = kwargs_pre.pop("border", False)
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, **kwargs_pre)
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label
    if border:
        fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
        fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    # things like `xaxis_tickmode` should be applied to all subplots. This is super janky lol but I'm under time pressure
    for setting in ["tickangle"]:
      if f"xaxis_{setting}" in kwargs_post:
          i = 2
          while f"xaxis{i}" in fig["layout"]:
            kwargs_post[f"xaxis{i}_{setting}"] = kwargs_post[f"xaxis_{setting}"]
            i += 1
    fig.update_layout(**kwargs_post)
    
    if return_fig:
      return fig
    else:
      fig.show(renderer=renderer)

def hist(tensor, renderer=None, **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    names = kwargs_pre.pop("names", None)
    if "barmode" not in kwargs_post:
        kwargs_post["barmode"] = "overlay"
    if "bargap" not in kwargs_post:
        kwargs_post["bargap"] = 0.0
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    fig = px.histogram(x=tensor, **kwargs_pre).update_layout(**kwargs_post)
    if names is not None:
        for i in range(len(fig.data)):
            fig.data[i]["name"] = names[i // 2]
    fig.show(renderer)


from plotly import graph_objects as go
from plotly.subplots import make_subplots




# code

In [28]:
model_name = "gpt2-small"
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed = True, 
    center_writing_weights = True,
    fold_ln = True, 
    refactor_factored_attn_matrices = False,
    device = device,
)
model.set_use_attn_result(True)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


In [29]:
owt_dataset = utils.get_dataset("owt")
BATCH_SIZE = 150
PROMPT_LEN = 50


all_owt_tokens = model.to_tokens(owt_dataset[0:BATCH_SIZE * 2]["text"]).to(device)
owt_tokens = all_owt_tokens[0:BATCH_SIZE][:, :PROMPT_LEN]
corrupted_owt_tokens = all_owt_tokens[BATCH_SIZE:BATCH_SIZE * 2][:, :PROMPT_LEN]
assert owt_tokens.shape == corrupted_owt_tokens.shape == (BATCH_SIZE, PROMPT_LEN)

logits, cache = model.run_with_cache(owt_tokens)

Found cached dataset openwebtext-10k (/data/cody_rushing/.cache/huggingface/datasets/stas___openwebtext-10k/plain_text/1.0.0/3a8df094c671b4cb63ed0b41f40fb3bd855e9ce2e3765e5df50abcdfb5ec144b)


In [30]:
def residual_stack_to_direct_effect(
    residual_stack: Float[Tensor, "... batch pos d_model"],
    effect_directions: Float[Tensor, "batch pos d_model"],
    apply_last_ln = True,
    scaling_cache = None
) -> Float[Tensor, "... batch pos_mins_one"]:
    '''
    Gets the direct effect towards a direction for a given stack of components in the residual stream.
    NOTE: IGNORES THE VERY LAST PREDICTION AND FIRST CLEAN TOKEN; WE DON'T KNOW THE ACTUAL PREDICTED ANSWER FOR IT!

    residual_stack: [... batch pos d_model] components of d_model vectors to measure direct effect from
    effect_directions: [batch pos d_model] vectors in d_model space that correspond to 'direct effect'
    scaling_cache = the cache to use for the scaling; defaults to the global clean cache
    '''
    if scaling_cache is None:
        raise ValueError("scaling_cache cannot be None")
    
    scaled_residual_stack = scaling_cache.apply_ln_to_stack(residual_stack, layer=-1, has_batch_dim=True) if apply_last_ln else residual_stack
    
    # remove the last prediction, and the direction of the zeroth token
    scaled_residual_stack = scaled_residual_stack[..., :, :-1, :]
    effect_directions = effect_directions[:, 1:, :]

    return einops.einsum(scaled_residual_stack, effect_directions, "... batch pos d_model, batch pos d_model -> ... batch pos")

In [31]:
def collect_direct_effect(cache, correct_tokens = owt_tokens, model = model):
    token_residual_directions: Float[Tensor, "batch seq_len d_model"] = model.tokens_to_residual_directions(correct_tokens)
    clean_per_head_residual: Float[Tensor, "head batch seq d_model"] = cache.stack_head_results(layer = -1, return_labels = False, apply_ln = False)
    per_head_direct_effect: Float[Tensor, "heads batch pos_minus_one"] = residual_stack_to_direct_effect(clean_per_head_residual, token_residual_directions, apply_last_ln = True, scaling_cache = cache)

    all_layer_output: Float[Tensor, "n_layer batch pos d_model"] = torch.zeros((model.cfg.n_layers, correct_tokens.shape[0], correct_tokens.shape[1], model.cfg.d_model)).cuda()
    for layer in range(model.cfg.n_layers):
        all_layer_output[layer, ...] = cache[f'blocks.{layer}.hook_mlp_out']

    all_layer_direct_effect = residual_stack_to_direct_effect(all_layer_output, token_residual_directions, apply_last_ln = True, scaling_cache = cache).cuda()
    return per_head_direct_effect, all_layer_direct_effect


per_head_direct_effect, all_layer_direct_effect = collect_direct_effect(cache)


In [32]:
def get_correct_logit_score(
    logits: Float[Tensor, "batch seq d_vocab"],
    clean_tokens: Float[Tensor, "batch 1"] = owt_tokens,
):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''
    smaller_logits = logits[:, :-1, :]
    smaller_correct = clean_tokens[:, 1:].unsqueeze(-1)
    answer_logits: Float[Tensor, "batch 2"] = smaller_logits.gather(dim=-1, index=smaller_correct)
    return answer_logits

In [33]:
def return_item(item):
    return item

new_logits = act_patch(model, owt_tokens, [Node("mlp_out", 11)], return_item, new_cache="zero", apply_metric_to_cache= False)
diff_in_logits = new_logits - logits
change_in_direct_effect = get_correct_logit_score(diff_in_logits)
    
direct_effects = all_layer_direct_effect[11].flatten().cpu()
assert direct_effects.shape == change_in_direct_effect.flatten().cpu().shape


#graphing:
    
fig = go.Figure()
text_labels = [f"Batch {i[0]}, Pos {i[1]}: {model.to_string(all_owt_tokens[i[0], i[1]:(i[1] + 1)])} --> {model.to_string(all_owt_tokens[i[0], (i[1] + 1):(i[1] + 2)])}" for i in itertools.product(range(BATCH_SIZE), range(PROMPT_LEN - 1))]
scatter_plot = go.Scatter(
    x = direct_effects,
    y = change_in_direct_effect.flatten().cpu(),
    text=text_labels,  # Set the hover labels to the text attribute
    mode='markers',
    marker=dict(size=2, opacity=0.8),
    name = "Change in Final Logits vs Direct Effect"
)
fig.add_trace(scatter_plot)
max_x = max(direct_effects.abs())

# add dashed y=-x line
fig.add_trace(go.Scatter(
    x=torch.linspace(-1 * max_x,max_x,100),
    y=torch.linspace(-1 * max_x,max_x,100) * -1,
    mode='lines',
    name='y = -x',
    line = dict(dash = 'dash')
))

fig.update_layout(
    title=f"Change in Final Logits vs Direct Effect for mlp11 in {model_name} for each Position and Batch. zero_ablated"    
)
fig.update_xaxes(title = f"Direct Effect of MLP11")
fig.update_yaxes(title = "Change in Final Logits")
fig.update_layout(width=900, height=500)
fig.show()